<a href="https://colab.research.google.com/github/lakshmi-keerthi/MLPractice/blob/main/Author_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

## Problem Statement

The problem is to identify the author of a  book from a given list of possible authors.

## Learning Objectives

At the end of the experiment, you will be able to do:

* Use NLTK package
* Handcraft features
* Preprocess the text
* Write an algorithm to identify author of a given book


## Background

Author identification is the task of identifying the author of a given text. It can be considered as a typical classification problem, where a set of books with known authors are used for training. The aim is to automatically determine the corresponding author of an anonymous text.

## Grading = 20 Marks

## Setup Steps

In [ ]:
#@title Run this cell to complete the setup for this Notebook

from IPython import get_ipython
ipython = get_ipython()

notebook="BLR_M1_Mini_Hackathon_AuthorIdentification" #name of the notebook
Answer = "This notebook is graded by mentors on the day of hackathon"
def setup():
#  ipython.magic("sx pip3 install torch")
   ipython.magic("sx wget https://www.dropbox.com/s/9xivz2pox1i83td/AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.bin?dl=1")
   ipython.magic("sx mv AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.bin?dl=1 AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.bin")
   print ("Setup completed successfully")
   return

setup()

Setup completed successfully


### NOTE: Please feel free to use ML libraries such as Sklearn, NLTK etc where ever applicable

### Downloading the required nltk Packages before moving ahead

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import nltk
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import gutenberg
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from urllib import request
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble, metrics, model_selection, naive_bayes
from sklearn.metrics import confusion_matrix
nltk.download('gutenberg')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
eng_stopwords = set(stopwords.words("english"))
wordnet = WordNetLemmatizer()

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## **Stage 1:** Sentence Level Classification


You can choose any two authors books from the below list

* Shakespeare
* Austen
* Chesterton

The books of the authors are mentioned in **Gutenberg Corpus**.

Click on the [link](https://www.nltk.org/book/ch02.html) for reference.

### 1 Mark ->  a) Access a particular book from the chosen authors. Find number of sentences and words (tokens) present in that book:

Instructions on accessing the Corpus can be found at the following [link](https://www.nltk.org/book/ch02.html)

In [ ]:
fileids = np.array(gutenberg.fileids())
for fileid in fileids:
  print("Book name           : ", fileid)
  number_of_words = gutenberg.words(fileids=fileid)
  print("Number of words     : ", len(number_of_words))
  number_of_sents = gutenberg.sents(fileids=fileid)
  print("Number of sentences : ", len(number_of_sents))
  print("\n")

Book name           :  austen-emma.txt
Number of words     :  192427
Number of sentences :  7752


Book name           :  austen-persuasion.txt
Number of words     :  98171
Number of sentences :  3747


Book name           :  austen-sense.txt
Number of words     :  141576
Number of sentences :  4999


Book name           :  bible-kjv.txt
Number of words     :  1010654
Number of sentences :  30103


Book name           :  blake-poems.txt
Number of words     :  8354
Number of sentences :  438


Book name           :  bryant-stories.txt
Number of words     :  55563
Number of sentences :  2863


Book name           :  burgess-busterbrown.txt
Number of words     :  18963
Number of sentences :  1054


Book name           :  carroll-alice.txt
Number of words     :  34110
Number of sentences :  1703


Book name           :  chesterton-ball.txt
Number of words     :  96996
Number of sentences :  4779


Book name           :  chesterton-brown.txt
Number of words     :  86063
Number of sentences 

### 2 Marks -> b) Ensure you appropriately split the multiple short stories for the below mentioned authors, Which will be your training data






**1.** Before moving ahead choose your two authors based on the your team-number: <br/>


Team=1,6,11,16,21,26 &nbsp;&nbsp;&nbsp;         Author-A Vs Author-B <br />
Team=2,7,12,17,22,27 &nbsp;&nbsp;&nbsp;         Author-B Vs Author-C <br />
Team=3,8,13,18,23,28 &nbsp;&nbsp;&nbsp;         Author-C Vs Author-D <br />
Team=4,9,14,19,24 &nbsp;&nbsp;&nbsp;            Author-D Vs Author-E <br />
Team=5,10,15,20,25&nbsp;&nbsp;&nbsp;            Author-E Vs Author-F <br />


**2.** Link for each of the author to be used for your problem: <br />



*   Author-A -> Rudyard Kipling ->  [Short Stories Collection](http://www.gutenberg.org/files/2781/2781-0.txt) &nbsp;&nbsp;
*   Author-B -> Anton Chekhov [Short Stories Collection](http://www.gutenberg.org/files/1732/1732-0.txt) &nbsp;&nbsp;
*   Author-C -> Guy De Maupassant [Short Stories Collection](http://www.gutenberg.org/cache/epub/21327/pg21327.txt)&nbsp;&nbsp;
*   Author-D -> O Henry [Short Stories Collection](http://www.gutenberg.org/cache/epub/2141/pg2141.txt) &nbsp;&nbsp;
*   Author-E -> Mark Twain [Short Stories Collection](http://www.gutenberg.org/files/245/245-0.txt)&nbsp;&nbsp;
*   Author-F -> Saki [Short Stories Collection](http://www.gutenberg.org/files/1477/1477-0.txt)&nbsp;&nbsp;

**Hint for downloading raw text from Gutenberg :**  Refer section "Electronic Books" in the following  [link](https://www.nltk.org/book/ch03.html) for the instructions.  





**Hint for splitting the multiple stories:**   You may use raw.find() and raw.rfind() in the same [link](https://www.nltk.org/book/ch03.html) to find appropriate index of the start and end location

In [ ]:
def split_into_multiple_stories(raw, stories, author):
  s_stories = []
  for index in range(len(stories)-1):
    short_story = []
    short_story.append(raw[raw.find(stories[index]):raw.rfind(stories[index+1])])
    short_story.append(author)
    s_stories.append(short_story)
  return s_stories

short_stories = []
A_url = "http://www.gutenberg.org/cache/epub/21327/pg21327.txt"
A_raw = request.urlopen(A_url).read().decode('utf8')
A_stories=['BOULE DE SUIF','MISS HARRIET',
     'FRANCESCA AND CARLOTTA RONDOLI', 'CHÂLI',
     'THE UMBRELLA', 'MY UNCLE SOSTHENES',
     'HE?', 'A PHILOSOPHER ',
     'ALWAYS LOCK THE DOOR!', 'A MEETING',
     'THE LITTLE CASK', 'HOW HE GOT THE LEGION OF HONOR ''THE ACCURSED BREAD''WHAT WAS REALLY THE MATTER WITH ANDREW ''MY LANDLADY ''THE HORLA, OR MODERN GHOSTS '
     'LOVE. THREE PAGES FROM A SPORTSMAN S BOOK ''SAVED  ''BELLFLOWER ''THE MARQUIS DE FUMEROL ''THE SIGNAL  ''THE DEVIL  ''EPIPHANY', 'IN THE WOOD ','A FAMILY','JOSEPH ',
     'THE INN ','UGLY','END OF THIS PROJECT GUTENBERG EBOOK']
A_raw = A_raw[A_raw.find('UGLY') + len('UGLY'):]
#print(A_raw)
short_stories.extend(split_into_multiple_stories(A_raw, A_stories, 'Guy De Maupassant'))




In [ ]:
B_url = "https://www.gutenberg.org/files/2781/2781-0.txt"
B_raw = request.urlopen(B_url).read().decode('utf8')
B_stories=['HOW THE WHALE GOT HIS THROAT', 'HOW THE CAMEL GOT HIS HUMP', 'HOW THE RHINOCEROS GOT HIS SKIN', 'HOW THE LEOPARD GOT HIS SPOTS', "THE ELEPHANT’S CHILD",
'THE SING-SONG OF OLD MAN KANGAROO','THE BEGINNING OF THE ARMADILLOS','HOW THE FIRST LETTER WAS WRITTEN','HOW THE ALPHABET WAS MADE','THE CRAB THAT PLAYED WITH THE SEA','THE CAT THAT WALKED BY HIMSELF','THE BUTTERFLY THAT STAMPED' ]
B_raw = B_raw[B_raw.find('THE BUTTERFLY THAT STAMPED') + len('THE BUTTERFLY THAT STAMPED'):]
#print(B_raw)
short_stories.extend(split_into_multiple_stories(B_raw, B_stories, 'Rudyard Kipling'))
#print(short_stories)

In [ ]:
dataset = pd.DataFrame(short_stories, columns = ['story', 'author'])
df = dataset.copy()
dataset['story'][0]
author_mapping_dict = {'Guy De Maupassant':0, 'Rudyard Kipling':1}
df['author'] = df['author'].map(author_mapping_dict)
label = df['author']

## **Stage 2**: Experiment with Handcrafted features representation


**Stylometry:**

Each person has their own unique vocabulary, sometimes rich, sometimes limited. Although a larger vocabulary is usually associated with literary quality, this is not always the case. Ernest Hemingway is famous for using a surprisingly small number of different words in his writing, which did not prevent him from winning the Nobel Prize for Literature in 1954.

Some people write in short sentences, while others prefer long blocks of text consisting of many clauses. No two people use semicolons, em-dashes, and other forms of punctuation in the exact same way.




**You may explore the following ways to analyze the text and generate handcrafted features by searching text in a probing way:**

a) Could  style of punctuation usage help as an handcrafted feature? Both by those who follow punctuations and by those who don't? Interesting [link](https://qwiklit.com/2014/03/05/top-10-authors-who-ignored-the-basic-rules-of-punctuation/)

b) The same word can sometimes be used in different contexts repeatedly by different authors. Could this fact be converted as a handcrafted feature? [link](https://www.nltk.org/book/ch01.html)

c) The above two are merely examples; As you might have noticed already the NLTK book [link](https://www.nltk.org/book/) offers several methods of analyzing and understanding text. Each of these analyses is in itself capable of being an handcrafted feature. **However for your evaluation a minimal set of useful handcrafted features which is helping you prove a classification of an is sufficient**

d) Could most command words be used to distinguish authors?  Refer "Counting Vocabulary" section of the [link](https://www.nltk.org/book/ch01.html)

e) How about using a count of most frequently used bi-gram, tri-grams and using it to classify an author?

f) How about using the frequency histogram of the most frequently used words across the stories by a given author a useful feature?

The limit here is endless limited only by your imagination, and of course your accuracy ! :)


### 2 Marks ->  a) List 6 handcrafted features to distinguish author stories.

In [ ]:
#A simple example:
#If a particular book by Shakespeare, has x unique words, y average sentence length then the feature vector for this author would look as follows:

1. Unique Words
2. AvgSentLength
3. Words
4. Characters
5. Stopwords
6. Punctations
7. UpperCaseWords
8. TitleCaseWords

###  4 Marks -> b) Write functions for any 4 of the above 6 features and label your authors accordingly.

In [ ]:
# UniqueWords    AvgSentLength     Label
#      x             y              1

## Number of words in the text
df["num_words"] = df["story"].apply(lambda x: len(str(x).split()))

## Number of unique words in the text ##
df["num_unique_words"] = df["story"].apply(lambda x: len(set(str(x).split())))

## Number of characters in the text ##
df["num_chars"] = df["story"].apply(lambda x: len(str(x)))

## Number of stopwords in the text ##
df["num_stopwords"] = df["story"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))

## Number of punctuations in the text ##
import string
df["num_punctuations"] =df['story'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )

## Number of title case words in the text ##
df["num_words_upper"] = df["story"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

## Number of title case words in the text ##
df["num_words_title"] = df["story"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

## Average length of the words in the text ##
df["mean_word_len"] = df["story"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

df


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


,story,author,num_words,num_unique_words,num_chars,num_stopwords,num_punctuations,num_words_upper,num_words_title,mean_word_len
0,"BOULE DE SUIF\r\n\r\n\r\nFor several days, str...",0,14580,4519,86645,6983,2574,74,1184,4.811797
1,MISS HARRIET\r\n\r\n\r\nThere were seven of us...,0,7847,2608,44077,3904,1462,231,675,4.478527
2,FRANCESCA AND CARLOTTA RONDOLI\r\n\r\n\r\nI\r\...,0,9902,2769,54334,5040,2086,356,1008,4.334175
3,"CHÂLI\r\n\r\n\r\nAdmiral de la Vallee, who see...",0,3756,1446,21140,1865,629,122,310,4.477103
4,THE UMBRELLA\r\n\r\n\r\nMme. Oreille was a ver...,0,3045,1061,16935,1550,696,63,283,4.380624
5,MY UNCLE SOSTHENES\r\n\r\n\r\nMy Uncle Sosthen...,0,5134,1648,27618,2714,997,274,525,4.234710
6,"HE?\r\n\r\n\r\nMy dear friend, you cannot unde...",0,66142,11309,365860,32891,15173,2039,6489,4.390977
7,,0,0,0,0,0,0,0,0,NaN
8,ALWAYS LOCK THE DOOR!\r\n\r\n\r\nThe four glas...,0,2098,905,11585,1067,500,83,190,4.391325
9,"A MEETING\r\n\r\n\r\nIt was all an accident, a...",0,3006,1111,16681,1545,631,60,260,4.395210


In [ ]:
#df.head()
df1 = df.copy()
X1 = df1.drop(['story','author','mean_word_len'], axis=1)

X1

,num_words,num_unique_words,num_chars,num_stopwords,num_punctuations,num_words_upper,num_words_title
0,14580,4519,86645,6983,2574,74,1184
1,7847,2608,44077,3904,1462,231,675
2,9902,2769,54334,5040,2086,356,1008
3,3756,1446,21140,1865,629,122,310
4,3045,1061,16935,1550,696,63,283
5,5134,1648,27618,2714,997,274,525
6,66142,11309,365860,32891,15173,2039,6489
7,0,0,0,0,0,0,0
8,2098,905,11585,1067,500,83,190
9,3006,1111,16681,1545,631,60,260


##**Stage 3:** Experiment with Text processing and representation:
TFIDF or CountVectorizer or Word2vec features



### 1 Mark -> a) Performing basic cleanup operations such as removing the newline characters and removing trailing spaces

**For example:** Your sentence looks as follows \[' This is a sentence\n\r. Another sentence \n'].

After newline removal from above example, your sentence will look like \['This is a sentence. Another sentence'] .

 In order to do this you can try using a combination of split() and join()

In [ ]:
df2 = dataset.copy()
df2 = df2.drop(['author'], axis=1)
df2['story'] = df2['story'].apply(lambda x: str(x).rstrip())
df2['story'] = df2['story'].apply(lambda x: " ".join(str(x).split()))
df2.head()

,story
0,"BOULE DE SUIF For several days, straggling rem..."
1,MISS HARRIET There were seven of us in a break...
2,FRANCESCA AND CARLOTTA RONDOLI I No (said my f...
3,"CHÂLI Admiral de la Vallee, who seemed to be h..."
4,THE UMBRELLA Mme. Oreille was a very economica...


### 1 Mark->b) Tokenize the sentence
**Example of a tokenized sentence:** <br/>Using same example from the previous cell for tokenization the sentence <br/> \['This is sentence a . Another sentence'].

Split above example into two sentences i.e. two elements in the list as follows:  \['This is a sentence .' , ' Another sentence']  <br/>

You may further choose to break sentences to individual words **if needed.**<br/>
**Example of sentences as list of words:**<br/>
**Before:** ['This is a sentence .' , ' Another sentence']<br/>
**After:** ['This', 'is' ,'a', 'sentence' , ' . ' , ' Another ', ' sentence ' ]

**Hint:** Consider using 'List comprehensions', whereby you split the sentence with spaces (using split(' ')) and populate the resulting output a list comprehension.



In [ ]:
df2['story'] = df2['story'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))
df2['story'] = df2['story'].apply(lambda x: x.lower())
df2['story'] = df2['story'].apply(lambda x: ' '.join([wordnet.lemmatize(word) for word in x.split() if not word in set(stopwords.words('english'))]))
df2.head()

,story
0,boule de suif several day straggling remnant r...
1,miss harriet seven u break four woman three me...
2,francesca carlotta rondoli said friend charles...
3,ch li admiral de la vallee seemed half asleep ...
4,umbrella mme oreille economical woman thorough...


###  1 Mark -> c) Is stop word removal necessary in the context of author identification? Your thoughts below?

In [ ]:
Yes

###  4 Marks-> d) Construct a labeled dataset of sentences

Create a representation of text, convert it into vectors (numbers). **Use any one** of the following algorithms for the task :

* Countvectorizer or
* TFIDFVectorizer or
* Word2Vec (The word2vec bin file (AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD) can be downloaded as a part of setup  )

References Documents:

1.   [Countvectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
2.  [TFIDFVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)


In [ ]:
#Define a CountVectorizer
cv = CountVectorizer()
#fit_transform your data to the vectorizer
X = cv.fit_transform(df2['story']).toarray()
X

array([[0, 1, 2, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
#Define a TfidfVectorizer
tfv = TfidfVectorizer()
#fit_transform your data to the vectorizer
X = tfv.fit_transform(df2['story']).toarray()
X

array([[0.        , 0.00504588, 0.01009177, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.01003929, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01039101, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
cv.get_feature_names()

['aback',
 'abandon',
 'abandoned',
 'abandoning',
 'abandonment',
 'abashed',
 'abated',
 'abb',
 'abbey',
 'abdullah',
 'abide',
 'abiding',
 'able',
 'abode',
 'abominable',
 'abominably',
 'abomination',
 'aboriginal',
 'abounds',
 'abraham',
 'abroad',
 'abrupt',
 'abruptly',
 'absence',
 'absent',
 'absolute',
 'absolutely',
 'absolution',
 'absorb',
 'absorbed',
 'absorbing',
 'abstemious',
 'abstraction',
 'absurd',
 'abuse',
 'abused',
 'abuts',
 'abyss',
 'academic',
 'academy',
 'acceded',
 'accent',
 'accentuate',
 'accentuated',
 'accept',
 'accepted',
 'accepting',
 'access',
 'accessed',
 'accessible',
 'accident',
 'accidentally',
 'acclamation',
 'accompanied',
 'accompany',
 'accomplice',
 'accomplish',
 'accomplished',
 'accord',
 'accordance',
 'accorded',
 'according',
 'accordingly',
 'accosted',
 'account',
 'accouterment',
 'accredited',
 'accrue',
 'accurately',
 'accursed',
 'accuse',
 'accused',
 'accuses',
 'accustomed',
 'achard',
 'ache',
 'ached',
 'ackno

In [ ]:
cv.inverse_transform(X[0])

[array(['abandon', 'abandoned', 'abated', ..., 'younger', 'yvetot', 'zero'],
       dtype='<U19')]

In [ ]:
df2['story'].iloc[0]

'boule de suif several day straggling remnant routed army passed town question organized troop simply disjointed rabble men unshaven dirty uniform tatter slouching along without regimental color without order worn broken incapable thought resolution marching pure habit dropping fatigue moment stopped majority belonged militia men peaceful pursuit retired tradespeople sinking weight accouterment quick witted little moblets prone terror enthusiasm ready attack fly red trouser remnant company mowed one big battle somber coated artilleryman side side various uniform infantry glittering helmet heavily booted dragoon followed difficulty march lighter footed soldier line company franc tireurs heroically named avenger defeat citizen tomb company death passed turn looking like horde bandit chief formerly draper corn dealer retired soap boiler suet refiner warrior circumstance created officer money length moustache heaped arm flannel gold lace talked loudly discussed plan campaign gave understan

AttributeError: ignored



##**Stage 4:** Classification :


### 4 Marks -> Perform a classification using either features obtained from Stage2 or Stage3

In [ ]:
#Spliting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=0.2, random_state=42)

#Training the model using KKN
MODEL_neigh = KNeighborsClassifier(n_neighbors=5)
MODEL_neigh.fit(X_train, y_train)

predicted_labels = MODEL_neigh.predict(X_test)
accuracy_score(y_test,predicted_labels)

0.6666666666666666

In [ ]:
#Model evaluation using confusion matrix
cm = confusion_matrix(y_test,predicted_labels)
cm

array([[4, 0],
       [2, 0]])

In [ ]:
#Training the model using Decision Tree
MODEL_decision_tree = DecisionTreeClassifier()
MODEL_decision_tree.fit(X_train,y_train)
MODEL_decision_tree.predict(X_test)

MODEL_decision_tree.score(X_test,y_test)

0.8333333333333334

# Further Ideas for exploration after the hackathon:

**Statistical analysis** of text using NLP, by analysis meaning of sentences, feature based grammars and analyzing structure of sentences! For all of this you can reference www.nltk.org/book